- hybrid engine 混合的是 inference（vllm/sglang） 和 training engine (fsdp/megtron-lm)
- Efficient actor model resharding with 3D-HybridEngine: Eliminates memory redundancy and significantly reduces communication overhead during transitions between **training and generation phases**.

### $\pi_\theta, \pi_{old}, \pi_{ref}$

- $\pi_{old}$ 并非一个独立持续存在的模型副本。在 PPO 中，它代表执行 Rollout (采样生成序列) 时所使用的策略模型。这通常是当前 $\pi_\theta$ 在该训练迭代开始时的状态。
- $\pi_{ref}$
    - `actor_rollout_ref.ref.fsdp_config.param_offload=True \`
    - 参数主要存储在 CPU内存中。当需要使用 π_ref 进行计算时（例如，计算 KL 散度所需的旧策略 log prob），FSDP 会按需将参数加载到 GPU 中进行计算。计算完成后，这些参数可以从 GPU 显存中释放，从而显著降低 π_ref 对 GPU 显存的常驻压力

### FSDPVLLMShardingManager

- `__enter__`:
    - 完成了从 FSDP (分片状态) 到 vLLM (完整状态) 的权重同步和加载，并进行了内存管理，使得 with 块内部可以安全地使用 vLLM 进行推理。
- ` __exit__`方法 (离开 with 块时执行)
    - 停用 vLLM，准备切换回 FSDP 训练模式。